# Textual Analysis
You should create an seperate `yaml` for each of the project you are working on, this is a good practice in general. Thesre migt be weird issues that causes a dependency problem. For the same reason a new `yaml` file would record all the dependencies that would ensure this pipeline works for future references.

In [16]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.keys()
pio.renderers.default = 'notebook' 

import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

from utils.eda import *
from utils.text_model import transform_text, tokenize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to /Users/kevinb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
# text data
fall_2022_text = pd.read_csv('data/2022_fall_text.csv')
winter_2023_text = pd.read_csv('data/2023_winter_text.csv')
spring_2023_text = pd.read_csv('data/2023_spring_text.csv')
summer_2023_text = pd.read_csv('data/2023_summer_text.csv')
fall_2023_text = pd.read_csv('data/2023_fall_text.csv')
winter_2024_text = pd.read_csv('data/2024_winter_text.csv')
spring_2024_text = pd.read_csv('data/2024_spring_text.csv')
summer_2024_text = pd.read_csv('data/2024_summer_text.csv')
fall_2024_text = pd.read_csv('data/2024_fall_text.csv')

In [18]:
# Initial Concat
text = pd.concat([fall_2022_text, winter_2023_text, spring_2023_text, summer_2023_text, fall_2023_text, winter_2024_text, spring_2024_text, summer_2024_text, fall_2024_text], axis=0)
clean_text = text.pipe(transform_text)
clean_text

,Quarter,Study Materials
0,2022Q1,"2022-02-14 | 2022Q1 | Book read (30m), DOC 2 P..."
1,2022Q1,"2022-02-15 | 2022Q1 | Nutrition read (15m), Ph..."
2,2022Q4 | 2022Q4,2022-10-21 | 2022Q4 | 2022Q4 |
3,2022Q4 | 2022Q4,2022-10-22 | 2022Q4 | 2022Q4 | After consecuti...
4,2022Q4 | 2022Q4 | 2022Q4 | 2022Q4 | 2022Q4,2022-10-23 | 2022Q4 | 2022Q4 | 2022Q4 | 2022Q4...
...,...,...
628,2024Q4,2024-12-11 | 2024Q4 | RPLH eval + paper algori...
629,2024Q4,"2024-12-12 | 2024Q4 | track-mjx meeting, RPLH ..."
630,2024Q4,"2024-12-13 | 2024Q4 | constraint article, EM a..."
631,2024Q4,2024-12-14 | 2024Q4 | chill


In [19]:
pre_process = (clean_text['Study Materials']
              .str.lower()
              .str.replace(r'\([\d]*m\)','',regex=True)
              .str.replace(',','')
              .str.strip())

corpus = ' '.join(pre_process.astype(str).to_list())
tokens = nltk.tokenize.word_tokenize(corpus, language='english')
len(tokens)

17285

# Making Chat Familier With My Data

Using either of `sentence-transformer`, `nltk`, `openai`, `langchain`, or related stuff has many dependency issue if just pyt in a big environment, need to have a seperated contained environment.

In [20]:
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import ast
import openai
import os

nltk.download('punkt')
model = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package punkt to /Users/kevinb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
from utils.text_model import split_text_nltk, get_similar_chunks, generate_response

documents = clean_text['Study Materials'].tolist()
all_chunks = []
for doc in documents:
    chunks = split_text_nltk(doc)
    all_chunks.extend(chunks)

print(f"Total chunks created: {len(all_chunks)}")
embeddings = model.encode(all_chunks, show_progress_bar=True, convert_to_tensor=False)

embedding_df = pd.DataFrame({
    'chunk': all_chunks,
    'embedding': embeddings.tolist(),
    'quarter': clean_text['Quarter']
})

embedding_df.to_csv('embeddings.csv', index=False)
print("Embeddings saved to embeddings.csv")

Total chunks created: 633


Batches: 100%|██████████| 20/20 [00:00<00:00, 37.33it/s]


Embeddings saved to embeddings.csv


In [25]:
embedding_df = pd.read_csv('embeddings.csv')
embedding_df['embedding'] = embedding_df['embedding'].apply(ast.literal_eval)

# Convert embeddings to a NumPy array of type float32
embeddings = np.array(embedding_df['embedding'].tolist()).astype('float32')

# Initialize FAISS index and using L2 distance, can also use cosine similarity
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings)
print(f"FAISS index has {index.ntotal} vectors.")

FAISS index has 633 vectors.


In [26]:
embedding_df

,chunk,embedding,quarter
0,"2022-02-14 | 2022Q1 | Book read (30m), DOC 2 P...","[-0.03888290748000145, 0.00010052329889731482,...",2022Q1
1,"2022-02-15 | 2022Q1 | Nutrition read (15m), Ph...","[-0.07007762789726257, -0.05126873403787613, 0...",2022Q1
2,2022-10-21 | 2022Q4 | 2022Q4 |,"[-0.04180379584431648, 0.03743821382522583, 0....",2022Q4 | 2022Q4
3,2022-10-22 | 2022Q4 | 2022Q4 | After consecuti...,"[-0.06285377591848373, -0.010527550242841244, ...",2022Q4 | 2022Q4
4,2022-10-23 | 2022Q4 | 2022Q4 | 2022Q4 | 2022Q4...,"[-0.022726742550730705, 0.037290364503860474, ...",2022Q4 | 2022Q4 | 2022Q4 | 2022Q4 | 2022Q4
...,...,...,...
628,2024-12-11 | 2024Q4 | RPLH eval + paper algori...,"[-0.07699394971132278, 0.07493709772825241, -0...",2024Q4
629,"2024-12-12 | 2024Q4 | track-mjx meeting, RPLH ...","[-0.028125612065196037, 0.027043156325817108, ...",2024Q4
630,"2024-12-13 | 2024Q4 | constraint article, EM a...","[-0.08803687244653702, 0.03149242326617241, -0...",2024Q4
631,2024-12-14 | 2024Q4 | chill,"[-0.06347204744815826, 0.03524710237979889, 0....",2024Q4


In [27]:
api_key = "..." #os.getenv('OPENAI_API_KEY')

In [29]:
user_prompt = "What did I mainly do in 2022 fall quarter?"
similar_chunks = get_similar_chunks(user_prompt, index, embedding_df, top_k=5)
generate_response(user_prompt, similar_chunks, api_key=api_key)

In [22]:
user_prompt = "What did I mainly do in 2023 fall quarter?"
similar_chunks = get_similar_chunks(user_prompt, index, embedding_df, top_k=5)
generate_response(user_prompt, similar_chunks, api_key=api_key)

"In 2023 fall quarter, I mainly did housework, planning, and training during the holiday break. I also spent time analyzing my spending and documenting it for Bank of America's 2022 fall quarter. I had a break on Christmas and New Year's Day. I also worked on various projects and submitted them, including a personal study time analysis project, a Cogs 9 final project, a DOC 1 portfolio, and a 大局观 for Math 18. I also had the opportunity to meet fun people during this time."

In [23]:
user_prompt = "What did I mainly do in 2024 fall quarter?"
similar_chunks = get_similar_chunks(user_prompt, index, embedding_df, top_k=5)
generate_response(user_prompt, similar_chunks, api_key=api_key)

'In 2024 fall quarter, you spent time cleaning your house, updating your resume and website, and analyzing study time data. You also spent time running and training your legs, and enjoyed some leisure activities. Additionally, you spent time working on a project for your Github repository.'